# Part 1: Creating Concurrent Processes

Objective: Create two concurrent processes that run in parallel and print numbers.

Steps:
1. Create a Python program where you spawn two processes.
2. Each process should print numbers from 1 to 5, with a delay between each print statement.
3. The output should show interleaving of the two processes' prints.

In [1]:
# Part 1: Creating Concurrent Threads
import threading
import time

def print_numbers(name, delay):
    for i in range(1, 6):
        print(f"Thread {name} printing {i}")
        time.sleep(delay)

thread0 = threading.Thread(target=print_numbers, args=(0, 0.02))
thread1 = threading.Thread(target=print_numbers, args=(1, 0.04))

thread0.start()
thread1.start()

thread0.join()
thread1.join()

Thread 0 printing 1
Thread 1 printing 1
Thread 0 printing 2
Thread 0 printing 3
Thread 1 printing 2
Thread 0 printing 4
Thread 0 printing 5
Thread 1 printing 3
Thread 1 printing 4
Thread 1 printing 5


# Part 2: Synchronization with Test_and_set

Objective: Demonstrate the use of Test_and_set to prevent race conditions when multiple processes or threads access a shared resource (e.g., a counter).

Steps:
1. Create a shared counter variable.
2. Spawn multiple threads or processes that increment the counter 100 times.
3. Without synchronization, print the counter value at the end to observe the race condition.
4. Implement Test_and_set to synchronize access to the shared counter and ensure the correct result.

In [2]:
import threading
import time

counter = 0

def increment_unsafe():
    global counter
    for _ in range(100):
        current = counter
        time.sleep(0.01) # workload
        counter = current + 1

lock = threading.Lock()

def increment_safe():
    global counter
    for _ in range(100):
        with lock:
            current = counter
            time.sleep(0.01) # workload
            counter = current + 1

def test(threads):
    global counter
    counter = 0

    for thread in threads:
        thread.start()
    for thread in threads:
        thread.join()

    print(f"Final counter value: {counter}")
    print(f"Expected value: 1000")


print("Testing without synchronization:")
test([threading.Thread(target=increment_unsafe) for _ in range(10)])

print("\nTesting with synchronization:")
test([threading.Thread(target=increment_safe) for _ in range(10)])

Testing without synchronization:
Final counter value: 100
Expected value: 1000

Testing with synchronization:
Final counter value: 1000
Expected value: 1000


# Part 3: Semaphores for Synchronization

Objective: Use semaphores to synchronize access to a shared resource (e.g., a printer or database).

Steps:
1. Create a semaphore initialized to 1 (binary semaphore).
2. Use the semaphore to control access to a critical section.
3. Spawn multiple processes or threads and use the semaphore to ensure mutual exclusion.

In [3]:
import threading
import time

semaphore = threading.Semaphore(1)
printer = []

def use_printer(thread_id):
    for i in range(3):
        semaphore.acquire()
        printer.append((thread_id, i))
        time.sleep(0.01) # workload
        semaphore.release()

threads = [threading.Thread(target=use_printer, args=(i,)) for i in range(3)]

for thread in threads:
    thread.start()
for thread in threads:
    thread.join()

for name, job in printer:
    print(f"Thread {name} used the printer for job {job}")


Thread 0 used the printer for job 0
Thread 0 used the printer for job 1
Thread 0 used the printer for job 2
Thread 1 used the printer for job 0
Thread 1 used the printer for job 1
Thread 1 used the printer for job 2
Thread 2 used the printer for job 0
Thread 2 used the printer for job 1
Thread 2 used the printer for job 2


# Part 4: Dining Philosophers Problem

Objective: Implement a basic version of the Dining Philosophers Problem where five philosophers try to eat by picking up two chopsticks.

Steps:
1. Create a program that simulates five philosophers sitting at a table.
2. Each philosopher alternates between thinking and eating.
3. Each philosopher needs to pick up two chopsticks before eating.
4. Without synchronization, observe the potential problems like deadlock or race conditions.
5. Try to implement a simple synchronization and observe the results.

In [4]:
import threading
import time

NUM_PHILOSOPHERS = 5
chopsticks = [threading.Lock() for _ in range(NUM_PHILOSOPHERS)]
printer = []

def philosopher(name):
    for i in range(3):
        printer.append(f"Philosopher {name} is thinking")
        time.sleep(0.01)
        left = chopsticks[name]
        right = chopsticks[(name + 1) % NUM_PHILOSOPHERS]

        # Pick up left then right chopstick
        left.acquire()
        right.acquire()
        printer.append(f"Philosopher {name} is eating meal {i}")
        time.sleep(0.2)
        right.release()
        left.release()
        printer.append(f"Philosopher {name} finished meal {i}")

threads = [threading.Thread(target=philosopher, args=(i,)) for i in range(NUM_PHILOSOPHERS)]

for thread in threads:
    thread.start()
for thread in threads:
    thread.join()

for output in printer:
    print(output)

Philosopher 0 is thinking
Philosopher 1 is thinking
Philosopher 2 is thinking
Philosopher 3 is thinking
Philosopher 4 is thinking
Philosopher 0 is eating meal 0
Philosopher 2 is eating meal 0
Philosopher 0 finished meal 0
Philosopher 0 is thinking
Philosopher 4 is eating meal 0
Philosopher 2 finished meal 0
Philosopher 2 is thinking
Philosopher 1 is eating meal 0
Philosopher 4 finished meal 0
Philosopher 4 is thinking
Philosopher 3 is eating meal 0
Philosopher 1 finished meal 0
Philosopher 1 is thinking
Philosopher 0 is eating meal 1
Philosopher 3 finished meal 0
Philosopher 3 is thinking
Philosopher 2 is eating meal 1
Philosopher 0 finished meal 1
Philosopher 0 is thinking
Philosopher 4 is eating meal 1
Philosopher 2 finished meal 1
Philosopher 2 is thinking
Philosopher 1 is eating meal 1
Philosopher 4 finished meal 1
Philosopher 4 is thinking
Philosopher 3 is eating meal 1
Philosopher 1 finished meal 1
Philosopher 1 is thinking
Philosopher 0 is eating meal 2
Philosopher 3 finished me